In [1]:
import requests
import pyspark
import pandas as pd
import mysql.connector as mydbconnection
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from sqlalchemy import create_engine
from mysql.connector import Error
from credentials import mysql_username, mysql_password

In [3]:
url = 'https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json'
response = requests.get(url)
print(f"Status code: {response.status_code}")

# Check if the request was successful
if response.status_code == 200:
    spark = SparkSession.builder.appName("API-loan-application").getOrCreate()

    # Convert the API response to a DataFrame
    df_loan_data = spark.read.json(spark.sparkContext.parallelize([response.json()]))
    print("Dataframe successfully created")
    df_loan_data.show()
else:
    print("Failed to fetch data from the API.")


Status code: 200
Dataframe successfully created
+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|Application_ID|Application_Status|Credit_History|Dependents|   Education|Gender|Income|Married|Property_Area|Self_Employed|
+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|      LP001002|                 Y|             1|         0|    Graduate|  Male|medium|     No|        Urban|           No|
|      LP001003|                 N|             1|         1|    Graduate|  Male|medium|    Yes|        Rural|           No|
|      LP001005|                 Y|             1|         0|    Graduate|  Male|   low|    Yes|        Urban|          Yes|
|      LP001006|                 Y|             1|         0|Not Graduate|  Male|   low|    Yes|        Urban|           No|
|      LP001008|                 Y|             1|         0|    Graduate|  M

In [4]:
# Fetch data from the API
'''url = 'https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json'
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    df_loan_app = pd.DataFrame(data)

    # Create a connection to the creditcard_capstone database
    database_username = mysql_username
    database_password = mysql_password
    database_ip = 'localhost'
    database_name = 'creditcard_capstone'
    database_connection = create_engine(f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}')

    # Load the data into the cdw_sapp_loan_application table
    df_loan_app.to_sql('cdw_sapp_loan_application', con=database_connection, if_exists='replace', index=False)

    print("Data successfully loaded into the database.")
else:
    print("Failed to fetch data from the API.")'''


'url = \'https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json\'\nresponse = requests.get(url)\n\n# Check if the request was successful\nif response.status_code == 200:\n    data = response.json()\n    df_loan_app = pd.DataFrame(data)\n\n    # Create a connection to the creditcard_capstone database\n    database_username = mysql_username\n    database_password = mysql_password\n    database_ip = \'localhost\'\n    database_name = \'creditcard_capstone\'\n    database_connection = create_engine(f\'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}\')\n\n    # Load the data into the cdw_sapp_loan_application table\n    df_loan_app.to_sql(\'cdw_sapp_loan_application\', con=database_connection, if_exists=\'replace\', index=False)\n\n    print("Data successfully loaded into the database.")\nelse:\n    print("Failed to fetch data from the API.")'

In [5]:
df_loan_data.columns
df_loan_data.printSchema()
df_loan_data.describe().show()
df_loan_data.count()

root
 |-- Application_ID: string (nullable = true)
 |-- Application_Status: string (nullable = true)
 |-- Credit_History: long (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Income: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Self_Employed: string (nullable = true)

+-------+--------------+------------------+------------------+------------------+------------+------+------+-------+-------------+-------------+
|summary|Application_ID|Application_Status|    Credit_History|        Dependents|   Education|Gender|Income|Married|Property_Area|Self_Employed|
+-------+--------------+------------------+------------------+------------------+------------+------+------+-------+-------------+-------------+
|  count|           511|               511|               511|               511|         511|   511|   511|    511|          511| 

511

In [6]:
# convert the PySpark DataFrame to a pandas DataFrame
df_loan_data = df_loan_data.toPandas()

In [7]:
loan_app_table = ( 
    "CREATE TABLE IF NOT EXISTS `cdw_sapp_loan_application` ("
    "Application_ID VARCHAR(10) PRIMARY KEY,"               
    "Application_Status VARCHAR(2),"               
    "Credit_History VARCHAR(2),"                   
    "Dependents VARCHAR(5),"                               
    "Education VARCHAR(15),"                        
    "Gender VARCHAR(10),"                           
    "Income VARCHAR(10),"                         
    "Married VARCHAR(10),"                          
    "Property_Area VARCHAR(20),"                   
    "Self_Employed VARCHAR(5)"                     
    ")"
)


In [8]:
try:
    conn = mydbconnection.connect(database='creditcard_capstone', user=mysql_username, password=mysql_password)
    cursor = conn.cursor()

    # Create the table
    print('Creating cdw_sapp_loan_application table....')
    cursor.execute(loan_app_table)
    print("cdw_sapp_loan_application table is created....")

    # Prepare the data for batch insertion
    data_to_insert = [tuple(row) for _, row in df_loan_data.iterrows()]
    sql = "INSERT INTO creditcard_capstone.cdw_sapp_loan_application VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

    # Batch insert the data
    cursor.executemany(sql, data_to_insert)
    print(f"All {len(data_to_insert)} Branch Records inserted")

    # Commit the transaction
    conn.commit()
    print("Branch data fully loaded")

except Error as e:
    print("Error while connecting to MySQL", e)

finally:
    # Ensure the connection is closed
    if conn and conn.is_connected():
        cursor.close()
        conn.close()
        print("MySQL connection is closed.")

Creating cdw_sapp_loan_application table....
cdw_sapp_loan_application table is created....
All 511 Branch Records inserted
Branch data fully loaded
MySQL connection is closed.


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 58443)
Traceback (most recent call last):
  File "c:\Users\fube9\AppData\Local\Programs\Python\Python310\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "c:\Users\fube9\AppData\Local\Programs\Python\Python310\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "c:\Users\fube9\AppData\Local\Programs\Python\Python310\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "c:\Users\fube9\AppData\Local\Programs\Python\Python310\lib\socketserver.py", line 747, in __init__
    self.handle()
  File "c:\Users\fube9\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\accumulators.py", line 281, in handle
    poll(accum_updates)
  File "c:\Users\fube9\AppData\Local\Programs\Python\Python310\lib